In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from pathlib import Path
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)
pd.set_option('display.float_format', '{:.6f}'.format)
start_time = time.time()

# Import Model

In [2]:
jupyter_dir = os.path.abspath(str(Path()))

In [3]:
models_path = os.path.join(jupyter_dir,'trained_models','trained_models','trained_models_old')
models = os.listdir(models_path)
for i, path in enumerate(models):
    print(f"{i}   {path}")

0   2024_10_07 17-55-55 Deep Neural Network 3oosRMSE
1   2024_10_08 00-00-41 Deep Neural Network 3oosRMSE
2   2024_10_08 00-24-59 Deep Neural Network 3oosRMSE
3   2024_10_08 00-42-22 Deep Neural Network 3oosRMSE
4   2024_10_09 09-01-43 Deep Neural Network 6oosRMSE
5   2024_10_09 09-08-27 Deep Neural Network 9oosRMSE
6   2024_10_09 09-11-43 Deep Neural Network 11oosRMSE
7   2024_10_09 10-09-20 Deep Neural Network 15oosRMSE
8   2024_10_09 11-11-13 Deep Neural Network 16oosRMSE


In [4]:
model = models[0]
model

'2024_10_07 17-55-55 Deep Neural Network 3oosRMSE'

In [5]:
model_path = os.path.join(models_path,model)

In [6]:
model_files = os.listdir(model_path)
model_files

['.ipynb_checkpoints',
 '2024_10_07 17-55-55 Deep Neural Network 3oosRMSE insample_results.csv',
 '2024_10_07 17-55-55 Deep Neural Network 3oosRMSE outofsample_results.csv',
 '2024_10_07 17-55-55 Deep Neural Network 3oosRMSE.pkl',
 '2024_10_07 17-55-55 Deep Neural Network 3oosRMSE.txt']

In [7]:
spec_path = os.path.join(model_path,[file for file in model_files if file.endswith('.txt')][0])

In [8]:
with open(spec_path, "r") as specs:
    for line in specs:
        print(line.strip())

Mon Oct  7 17:49:14 2024
spot(s):
[1311.13 1325.42 1330.53 1333.44 1338.73 1351.52 1372.91 1374.2  1377.99
1380.79 1386.77 1386.98 1387.16 1390.08 1391.82 1392.05 1395.15 1399.96
1401.2  1401.5  1401.77 1402.74 1403.05 1406.48 1406.58 1406.64 1406.77
1409.11 1409.82 1410.74 1411.59 1412.12 1412.71 1413.18 1414.71 1416.12
1416.34 1416.4  1416.59 1416.97 1418.28 1419.95 1420.46 1420.83 1422.42
1422.43 1423.07 1423.8  1424.27 1424.44 1426.09 1426.6  1428.02 1428.35
1428.7  1428.81 1430.36 1430.59 1430.69 1431.78 1433.18 1433.19 1433.55
1433.65 1434.56 1434.69 1436.11 1437.38 1437.45 1438.07 1438.45 1438.91
1438.99 1439.15 1439.53 1439.88 1439.97 1443.66 1444.24 1444.47 1445.28
1445.5  1445.71 1446.04 1446.9  1447.03 1447.04 1447.54 1447.77 1447.89
1448.38 1448.46 1449.6  1449.93 1451.21 1451.23 1451.6  1452.52 1452.78
1452.95 1452.96 1453.53 1453.85 1454.4  1454.47 1454.78 1455.05 1455.51
1456.3  1456.93 1457.51 1457.85 1458.35 1459.18 1459.53 1460.37 1462.63
1462.85 1463.59 1463.78 1466.

In [9]:
model_path

'E:\\git\\machine-learning-option-pricing\\trained_models\\trained_models\\trained_models_old\\2024_10_07 17-55-55 Deep Neural Network 3oosRMSE'

In [10]:
model_files

['.ipynb_checkpoints',
 '2024_10_07 17-55-55 Deep Neural Network 3oosRMSE insample_results.csv',
 '2024_10_07 17-55-55 Deep Neural Network 3oosRMSE outofsample_results.csv',
 '2024_10_07 17-55-55 Deep Neural Network 3oosRMSE.pkl',
 '2024_10_07 17-55-55 Deep Neural Network 3oosRMSE.txt']

In [11]:
outsample_path = os.path.join(model_path,[file for file in model_files if file.find('outofsample') != -1][0])
outsample_path

'E:\\git\\machine-learning-option-pricing\\trained_models\\trained_models\\trained_models_old\\2024_10_07 17-55-55 Deep Neural Network 3oosRMSE\\2024_10_07 17-55-55 Deep Neural Network 3oosRMSE outofsample_results.csv'

In [ ]:
outsample_path = os.path.join(model_path,[file for file in model_files if file.find('insample') != -1][0])
outsample = pd.read_csv(outsample_path).iloc[:,1:]
outsample

# Test Model

In [ ]:
maturities = outsample['days_to_maturity'].unique().tolist()
barrier_type_names = outsample['barrier_type_name'].unique().tolist()
ws = outsample['w'].unique().tolist()
print(f"\nmaturities:\n     maturities = {maturities}")
print(f"\ntypes:\n     barrier_type_names = {barrier_type_names}\n     ws = {ws}\n")

In [ ]:
by_wbt = outsample.copy().set_index(['w','barrier_type_name','days_to_maturity']).sort_index()

In [ ]:
RMSE = pd.DataFrame(
    np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
    columns = barrier_type_names,
    index = maturities
)

for t in maturities:
    for b in barrier_type_names:
        RMSE.loc[t,b] = np.sqrt(np.average((by_wbt.loc[(slice(None), b, t),'outofsample_error'])**2))
print(f"\nRMSE:\n{RMSE}")

In [ ]:
MAE = pd.DataFrame(
    np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
    index = maturities,
    columns = barrier_type_names
)
for t in maturities:
    for b in barrier_type_names:
        MAE.loc[t,b] = np.average(np.abs(by_wbt.loc[(slice(None), b, t),'outofsample_error']))
print(f"\nMAE:\n{MAE}")

In [ ]:
plt.figure()
for b in barrier_type_names:
    plt.plot(MAE.loc[:, b], label=b)
plt.xlabel('maturities in days')
plt.ylabel('RMSE')
plt.legend()
plt.show()
plt.close()

In [ ]:
errors_bypayoff = {}
for w in ws:
    errors = {}
    print(
        f"\n         {'#'*(len(w)+5)}"
        f"\n         # {w}s #"
        f"\n         {'#'*(len(w)+5)}"
    )
    RMSE = pd.DataFrame(
        np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
        columns = barrier_type_names
        )
    RMSE[w] = maturities
    RMSE = RMSE.set_index(w,drop=True)
    for t in maturities:
        for b in barrier_type_names:
            RMSE.loc[t,b] = np.sqrt(np.average((by_wbt.loc[(w, b, t),'outofsample_error'])**2))
    print(f"RMSE:\n{RMSE}")
    errors['RMSE'] = RMSE
    


    MAE = pd.DataFrame(
        np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
        columns = barrier_type_names
    )
    MAE[w] = maturities
    MAE = MAE.set_index(w,drop=True)
    for t in maturities:
        for b in barrier_type_names:
            MAE.loc[t,b] = np.average(np.abs(by_wbt.loc[(w, b, t),'outofsample_error']))
    print(f"MAE:\n{MAE}")
    errors['MAE'] = MAE

    errors_bypayoff[w] = errors
    print("\n\n")

In [ ]:
for w, errors in errors_bypayoff.items():
    for b in barrier_type_names: 
        print(errors['MAE'])

In [ ]:
plt.figure()
for w, errors in errors_bypayoff.items():
    for b in barrier_type_names: 
        MAE = errors['MAE'][b]
        plt.plot(
            MAE.sort_values(), label=f"{b} {w}s ",
        )
plt.legend(loc='best')
plt.show()
plt.close()

In [ ]:
end_time = time.time()
runtime = end_time - start_time
print(f"cpu: {runtime}")